In [438]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn as sk
import seaborn as sns
import math
from mlxtend.feature_selection import SequentialFeatureSelector as sfs





# Colunas

school - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)

sex - student's sex (binary: 'F' - female or 'M' - male)

age - student's age (numeric: from 15 to 22).

~~address - student's home address type (binary: 'U' - urban or 'R' - rural)~~

~~famsize - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3).~~

Pstatus - parent's cohabitation status (binary: 'T' - living together or 'A' - apart).

Medu - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education).

Fedu - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education).

~~Mjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other').~~

~~Fjob - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')~~

~~reason - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other').~~

~~guardian - student's guardian (nominal: 'mother', 'father' or 'other').~~

~~traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)~~

studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours).

failures - number of past class failures (numeric: n if 1<=n<3, else 4)

schoolsup - extra educational support (binary: yes or no)

famsup - family educational support (binary: yes or no)

paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)

activities - extra-curricular activities (binary: yes or no)

~~nursery - attended nursery school (binary: yes or no)~~

higher - wants to take higher education (binary: yes or no)

internet - Internet access at home (binary: yes or no)

romantic - with a romantic relationship (binary: yes or no)

~~famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)~~

freetime - free time after school (numeric: from 1 - very low to 5 - very high)

goout - going out with friends (numeric: from 1 - very low to 5 - very high)

Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)

Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)

health - current health status (numeric: from 1 - very bad to 5 - very good)

absences - number of school absences (numeric: from 0 to 93)


In [439]:
#Importar dados do csv
df = pd.read_csv('student_data.csv')
#df_ss = pd.read_csv('sample_submission.csv')
#array = df_ss.iloc[:,1:].values
#df.insert(0,'G3', array)
df = df.drop(['index'], axis=1)
print(df)

     age  Medu  Fedu  traveltime  studytime  failures  famrel  freetime  \
0     15     4     2           1          1         0       3         5   
1     18     2     1           2          2         0       4         3   
2     17     3     3           2          4         1       5         4   
3     16     3     3           1          3         0       4         3   
4     17     1     1           2          1         0       4         4   
..   ...   ...   ...         ...        ...       ...     ...       ...   
540   17     4     4           3          1         3       3         3   
541   17     1     1           1          3         0       4         3   
542   17     1     3           2          1         0       5         1   
543   17     1     0           2          2         0       4         4   
544   17     2     2           1          1         0       5         1   

     goout  Dalc  ...  activities_no  activities_yes  nursery_no  nursery_yes  \
0        2     1  

In [440]:
#Dividir em X e y
X = df.drop(['G3'], axis=1)
y = df['G3']

In [441]:
#Funcao para calcular RMSE
def rmse(y, y_pred):
    mse = sk.metrics.mean_squared_error(y, y_pred)
    rmse = math.sqrt(mse)
    f = '{0:.3g}'.format(rmse)
    print(f"RMSE: {f}")

In [442]:
#Eliminar Colunas com o metodo Backwards
# lreg = LinearRegression()

# sfs1 = sfs(lreg, k_features=50, forward=False, verbose=0, scoring='neg_mean_squared_error')
# sfs1 = sfs1.fit(X, y)
# drop_columns = list(sfs1.k_feature_names_)
# new_df = df.drop(drop_columns, axis = 1)
# print(new_df)

In [443]:
#Dividir o novo dataframe em dois, um de teste e um de treino (20% / 80%)
df_train, df_test = train_test_split(df, train_size = 0.8, test_size = 0.2, random_state = 0)

In [444]:
def build_model(X,y):
    X = sm.add_constant(X) #Adding the constant
    lm = sm.OLS(y,X).fit() # fitting the model
    print(lm.summary()) # model summary
    return X
    
def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)


In [445]:
#Usar o novo DF sem as colunas que nao interessam, dividir
X = df.drop(['G3'], axis=1)
y = df['G3'].values

from sklearn.model_selection import train_test_split
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


X_train_new = build_model(X_train, y_train)
X_train_new = X_train_new.drop('const', axis=1)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.395
Model:                            OLS   Adj. R-squared:                  0.326
Method:                 Least Squares   F-statistic:                     5.710
Date:                Fri, 17 Jun 2022   Prob (F-statistic):           4.86e-20
Time:                        12:21:18   Log-Likelihood:                -890.43
No. Observations:                 381   AIC:                             1861.
Df Residuals:                     341   BIC:                             2019.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.9217      0.29

In [446]:
# lreg = LinearRegression()
# sfs1 = sfs(lreg, k_features=20, forward=False, verbose=0, scoring='neg_mean_squared_error')
# sfs1 = sfs1.fit(X, y)
# feat_names = list(sfs1.k_feature_names_)
# print(feat_names)

# not_drop = feat_names

# not_drop = ['failures', 'school_GP', 'schoolsup_no', 'higher_yes','internet_yes','Mjob_health','sex_F',
# 'studytime','famsize_GT3','famsize_LE3','Mjob_health','Fjob_health','Fjob_teacher','freetime','goout','reason_reputation','guardian_other','paid_no','nursery_yes',
# 'G3']

# not_drop = [ 'school_GP', 'schoolsup_no', 'higher_yes','internet_yes',
# 'studytime','famsize_GT3','famsize_LE3','Mjob_health','Fjob_health','Fjob_teacher','freetime','goout','reason_reputation','guardian_other','paid_no','nursery_yes',
# 'G3']

not_drop = ['studytime','failures','school_MS','Pstatus_A','Mjob_health','Mjob_teacher','Fjob_health','Fjob_teacher','reason_reputation','schoolsup_no','paid_yes','nursery_no','G3']


In [447]:
#Dropar colunas e refazer o modelo
#drops = ["Fjob_at_home", "famsup_no", "Fjob_services", "Fjob_teacher", "reason_course", "reason_home", "reason_other", "reason_reputation", "guardian_father", "guardian_mother", "guardian_other",
# "schoolsup_no", "schoolsup_yes", "famsup_yes", "Fjob_health", "paid_no", "paid_yes", "activities_no", "activities_yes", "nursery_no", "nursery_yes", "higher_no", "higher_yes", "internet_no",
#  "internet_no", "romantic_no", "Fjob_other", "romantic_yes", "school_MS", "famsize_LE3", "school_GP", "Mjob_teacher", "sex_F", "sex_M", "address_R", "address_R", "famsize_GT3", "Pstatus_A",
#  "Pstatus_T", "Mjob_at_home", "Mjob_health", "Mjob_other", "Mjob_services"]

drops= []
# not_drop = ["romantic_no",
# "health",
# #"internet_yes",
# "higher_yes",
# #"famsup_yes",
# "famsup_no",
# "schoolsup_no",
# #"famsize_LE3",
# "address_U",
# #"sex_F",
# "school_GP",
# "failures", 
# "studytime", 
# #"Pstatus_T", 
# #"guardian_father", 
# #"guardian_mother"
# "activities_yes"
# ]

for c in X_train_new.columns:
    if c not in not_drop:
        drops.append(c)
            
X_train_new = X_train_new.drop(drops, axis=1)

X_train_new = build_model(X_train_new, y_train)
X_train_new = X_train_new.drop("const", axis=1)

vif = checkVIF(X_train_new)
print(vif)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.271
Method:                 Least Squares   F-statistic:                     12.76
Date:                Fri, 17 Jun 2022   Prob (F-statistic):           6.95e-22
Time:                        12:21:18   Log-Likelihood:                -919.89
No. Observations:                 381   AIC:                             1866.
Df Residuals:                     368   BIC:                             1917.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 9.9935      0.61

In [448]:
lm = sm.OLS(y_train,X_train_new).fit()
y_train_score = lm.predict(X_train_new)

In [449]:
# Now let's use our model to make predictions.
# Creating X_test_new dataframe by dropping variables from X_test
X_test_new = X_test[X_train_new.columns]

# Adding a constant variable 
#X_test_new = sm.add_constant(X_test_new)

In [450]:



# Making predictions
y_pred = lm.predict(X_test_new)


y_pred_final = []

# Passar os valores para inteiro pois a nota do G3 neste dataset está como inteiro
for c in y_pred:
    y_pred_final.append(int(round(c)))


y_pred_final = np.array(y_pred_final)
#print(y_pred_final)


rmse(y_test, y_pred_final) #+ 0.7



RMSE: 3.43


In [451]:
print(y_test)

print('Predict')

print(y_pred_final)

[15 11 14 11 10  7 11 11  9 10 15  0 15 13 14 11 15  9 13  8 14 16 12 13
  0 10 17 13 17 11 10 13  8 16  8 10 14 11  9 11 15  9 11 12 11 10 10 17
 11 12 12  0 14 13 15 11 13 16  8 15 16 14 10 15 13 12 12  8 13 13  9 10
 15 10 15  9 15  0 10 11 15 13 10  8 10 13  9 16 10 13 10 11 12  9 13 10
  7  9 11 11 11  8 17 10 12 10 12 13 11 12 13 10 16 11  9 11 11 14 10 14
 15  7 13 14 12 14 10  8 12 13 10 10 14 11 10 10 13 10 11 13 10 10  9 11
 11 10 13 13  9 16 12 11  9 11 15 16 10 15 11 17 14 13 15 10]
Predict
[16 11  9  9 11  9  9  7 12 12 13  8 10 12 12  9 12  9 12  9 15 12 19 13
  9  9 13 12 15 11  9 12  9 10 11  8 12  4 12  7 17  9 12  8  9  9 10 12
 12 12 14 12 17 12 13 14 12 12 16 16 13  9  8 12 14 12 10 13 12 10  9 11
 17  9 16 10 14  8 11  7 13 16 12  9 10 14 13  3 14 14 13 18 14  9 13 13
 11  9 10 10 12 12 19  9  5 14 12 14  6  9  8 14 18 14  4 11 11 11 12 15
 10  8 12 12  6 12 10  9  9 13  9 10 14 13 10 11 13 14  9 10 13  8 10  9
 10 13  7  5 15 15 13 10 13  9 12  9  5 13 10 16 13  7

In [452]:
#Prever os resultados finais
#Importar dados do csv
df_final = pd.read_csv('test_data.csv')
df_final = df_final.drop(drops, axis=1)
df_final = df_final.drop(["index"], axis=1)

pred_final = lm.predict(df_final)

print(pred_final)

0      15.753429
1      15.211411
2      12.812508
3       9.157308
4      13.354527
         ...    
99      6.717395
100     7.770322
101    18.152332
102    17.360549
103    13.576338
Length: 104, dtype: float64


In [453]:
#Export
d = {'index': [x for x in range(0,104)], 'G3': pred_final}
df_export = pd.DataFrame(data=d)  
df_export = df_export.reset_index(drop=True)
#first_column = df_export.columns[0]
# Delete first
#df_export = df_export.drop([first_column], axis=1)

#Export
df_export.to_csv('output.csv', sep=',', index=False)

print(df_export)

     index         G3
0        0  15.753429
1        1  15.211411
2        2  12.812508
3        3   9.157308
4        4  13.354527
..     ...        ...
99      99   6.717395
100    100   7.770322
101    101  18.152332
102    102  17.360549
103    103  13.576338

[104 rows x 2 columns]
